In [2]:
import pandas as pd

# download url data from internet
import requests
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text   
from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,'lxml')

In [3]:
table = soup.find('table')

In [4]:
column_names=['Postcode','Borough','Neighbourhood']
df = pd.DataFrame(columns=column_names)

In [5]:
# extracting information from the table
for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data

In [6]:
# remove rows where Borough is 'Not assigned'
df=df[df['Borough']!='Not assigned']

# assign Neighbourhood=Borough where Neighbourhood is 'Not assigned'
df[df['Neighbourhood']=='Not assigned']=df['Borough']


# group multiple Neighbourhood under one Postcode
temp_df=df.groupby('Postcode')['Neighbourhood'].apply(lambda x: "%s" % ', '.join(x))
temp_df=temp_df.reset_index(drop=False)
temp_df.rename(columns={'Neighbourhood':'Neighbourhood_joined'},inplace=True)

# join the newly constructed joined data frame
df_merge = pd.merge(df, temp_df, on='Postcode')

# drop the Neighbourhood column
df_merge.drop(['Neighbourhood'],axis=1,inplace=True)

# drop duplicates from the data frame
df_merge.drop_duplicates(inplace=True)


# rename Neighbourhood_joined back to Neighbourhood
df_merge.rename(columns={'Neighbourhood_joined':'Neighbourhood'},inplace=True)

In [7]:
df_merge.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
4,M6A,North York,"Lawrence Heights, Lawrence Manor"
6,Queen's Park,Queen's Park,Queen's Park


In [8]:
df_merge.shape

(103, 3)